In [5]:
# from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options
import os
import json
import re
import datetime
import sys
import pandas as pd

try: 
    from pyvirtualdisplay import Display
except  ModuleNotFoundError:
    !pip install PyVirtualDisplay
    
try: 
    from seleniumwire import webdriver  # Import from seleniumwire
except  ModuleNotFoundError:
    print("module selenium-wire is not installed, now start installation.")
    !pip install selenium-wire

In [6]:
# For mouse scroll
def wheel_element(element, deltaX = 0, deltaY = 120, offsetX = 0, offsetY = 0):
    error = element._parent.execute_script("""
    var element = arguments[0];
    var deltaX = arguments[1]
    var deltaY = arguments[2];
    var box = element.getBoundingClientRect();
    var clientX = box.left + (arguments[3] || box.width / 2);
    var clientY = box.top + (arguments[4] || box.height / 2);
    var target = element.ownerDocument.elementFromPoint(clientX, clientY);

    for (var e = target; e; e = e.parentElement) {
      if (e === element) {
        target.dispatchEvent(new MouseEvent('mouseover', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new MouseEvent('mousemove', {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY}));
        target.dispatchEvent(new WheelEvent('wheel',     {view: window, bubbles: true, cancelable: true, clientX: clientX, clientY: clientY, deltaX: deltaX, deltaY: deltaY}));
        return;
      }
    }    
    return "Element is not interactable";
    """, element, deltaX, deltaY, offsetY)

def get_websocket_data(ticker, interval, monitor=True):
    # Initiate a chrome web driver
    chrm_options = Options()
    chrm_caps = webdriver.DesiredCapabilities.CHROME.copy()
    chrm_caps['goog:loggingPrefs'] = { 'performance':'ALL' } #save all returns including websocket data
    
    if monitor == False:
        display = Display(visible=0, size=(1024, 768)) 
        display.start()
    
    driver = webdriver.Chrome(chrome_options=chrm_options,
                              desired_capabilities=chrm_caps) 

    driver.maximize_window()
    # Go to Trading View
    website = 'https://www.tradingview.com/'
    driver.get(website)
    print('Reached tradingview website')
    
    # Input query
    driver.find_element_by_tag_name('body').send_keys(Keys.COMMAND + 'k')
    driver.implicitly_wait(3)

    query = driver.find_element_by_name('query')
    query.send_keys(ticker)
    print('Typed in ticker name')
    driver.implicitly_wait(3)

#     query.submit()
    query.send_keys(Keys.ENTER)
    print('Pressed search')
    driver.implicitly_wait(3)
    
    # Select frequency
    control = driver.find_element_by_id('header-toolbar-intervals')
    control.click()
    
    if interval == "1h":
        select_1h = driver.find_element_by_xpath("//div[@data-value='60']")
        select_1h.click()
        print('Selected 1h interval')

#         driver.refresh()
        
    # Scroll the chart to retrieve all data    
    elm = driver.find_element_by_class_name('chart-gui-wrapper')
    for i in range(500):
        wheel_element(elm, deltaX=0, deltaY=100)
    print('Zoomed out')

    for i in range(5000):
        wheel_element(elm, deltaX=-10, deltaY=0)
    print('Zoomed left')
    driver.close()
    print('Done!')
    
    if monitor == False:
        display.end()
        
    return driver.requests

def parse_driver_requests(driver_requests):
    ws = []
    for request in driver_requests:
        if request.ws_messages:
            ws.append(request.ws_messages)
            
    messages = []
    for w in ws:
        for l in w:
            m = l.content
            txt_list = re.split('~m~\d*~m~',m)       
            for t in txt_list:
                if len(t)<10:
                    continue
                t = json.loads(t)
                try:
                    if t['m'] == 'timescale_update':
                        messages.append(t)
                except:
                    continue
                    
    price_data = []

    for m in messages:
        try:
            _j = m['p'][1]
            price_data.append([s['v']  for s in _j['sds_1']['s']])
        except:
            continue
            
    if len(price_data)==0:
        print('ticker not avaliable')
        return None
    df = [pd.DataFrame(p) for p in price_data]
    df = pd.concat(df)

    df.columns = ['Date', 'open', 'high', 'low', 'close', 'volume']
    df.Date = pd.to_datetime(df.Date, unit='s')
    
    print('before drop duplicate', len(df))
    df = df.drop_duplicates('Date', keep='last')
    df.sort_values('Date', inplace=True)
    print('after',len(df))
    df.reset_index(inplace=True, drop=True)
    return df

In [11]:
# driver_requests = get_websocket_data('ES1!', '1h', True)
query = 'BITMEX:BTCUSD'
driver_requests = get_websocket_data(query, '1h', True)
data = parse_driver_requests(driver_requests)

Reached tradingview website
Typed in ticker name
Pressed search
Selected 1h interval
Zoomed out
Zoomed left
Done!
before drop duplicate 367
after 367


In [12]:
data

,Date,open,high,low,close,volume
0,2021-03-07 23:00:00,3854.75,3878.75,3796.25,3819.25,2198363.0
1,2021-03-08 23:00:00,3831.25,3901.25,3826.25,3873.25,1704837.0
2,2021-03-09 23:00:00,3878.50,3916.25,3856.25,3896.50,1845205.0
3,2021-03-10 23:00:00,3907.75,3958.50,3893.00,3936.75,1599748.0
4,2021-03-11 23:00:00,3928.00,3938.50,3900.75,3932.75,1596215.0
...,...,...,...,...,...,...
362,2022-08-11 22:00:00,4219.25,4282.75,4208.25,4281.00,1227330.0
363,2022-08-14 22:00:00,4277.00,4304.75,4249.00,4298.25,1279387.0
364,2022-08-15 22:00:00,4294.50,4327.50,4278.75,4307.75,1322087.0
365,2022-08-16 22:00:00,4309.25,4315.00,4255.00,4276.75,1588532.0
